# This is a learning project. It aims to have hands on experience in NLP

### Trying to build a model that labels the customer complaints
### Will work with [Automatic Ticket Classification](https://www.kaggle.com/datasets/venkatasubramanian/automatic-ticket-classification) from Kaggle

In [5]:
import pandas as pd
import numpy as np
import os
import json

In [6]:
data_dir = 'data/all_tickets.csv'

In [7]:
customer_tickets = pd.read_csv(data_dir)

In [8]:
customer_tickets.head()

,title,body,ticket_type,category,sub_category1,sub_category2,business_service,urgency,impact
0,NaN,hi since recruiter lead permission approve req...,1,4,2,21,71,3,4
1,connection with icon,icon dear please setup icon per icon engineers...,1,6,22,7,26,3,4
2,work experience user,work experience user hi work experience studen...,1,5,13,7,32,3,4
3,requesting for meeting,requesting meeting hi please help follow equip...,1,5,13,7,32,3,4
4,reset passwords for external accounts,re expire days hi ask help update passwords co...,1,4,2,76,4,3,4


Will use the ``body`` to predict the ``category`` 

In [9]:
columns = list(customer_tickets.columns)
print(columns)

['title', 'body', 'ticket_type', 'category', 'sub_category1', 'sub_category2', 'business_service', 'urgency', 'impact']


In [10]:
customer_tickets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48549 entries, 0 to 48548
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   title             47837 non-null  object
 1   body              48549 non-null  object
 2   ticket_type       48549 non-null  int64 
 3   category          48549 non-null  int64 
 4   sub_category1     48549 non-null  int64 
 5   sub_category2     48549 non-null  int64 
 6   business_service  48549 non-null  int64 
 7   urgency           48549 non-null  int64 
 8   impact            48549 non-null  int64 
dtypes: int64(7), object(2)
memory usage: 3.3+ MB


In [12]:
print("Ticket category unique values: ", customer_tickets['category'].nunique())
print("row count: ", customer_tickets.shape[0])

Ticket category unique values:  13
row count:  48549
